In [1]:
import torch
import torch.nn as nn
from sentiment_preprocessing import SentimentAnalysisDataset, EmbeddingDataset
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
torch.random.manual_seed(13)
import sys
sys.path.append('..')
from ELMO import ELMo
from sentiment_models import SentimentAnalysis, SentimentAnalysis_WithoutELMo
import wandb

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
path = 'Marathi_Semantic/pos_train.txt'
with open (path, "r", encoding='utf-8') as f:
    pos_train = f.readlines()
path = 'Marathi_Semantic/neg_train.txt'
with open (path, "r", encoding='utf-8') as f:
    neg_train = f.readlines()

In [4]:
# make a tensor of labels
y_pos = torch.ones(len(pos_train), dtype=torch.long)
y_neg = torch.zeros(len(neg_train), dtype=torch.long)
labels = torch.cat((y_pos, y_neg), dim=0)

In [5]:
data = pos_train + neg_train

In [6]:
char_vocab = torch.load('../ELMo/char_vocab_marathi.pt')
word_vocab = torch.load('../ELMo/word_vocab_marathi.pt')

In [7]:
elmo = ELMo(cnn_config = {'character_embedding_size': 16, 
                           'num_filters': 32, 
                           'kernel_size': 5, 
                           'max_word_length': 10, 
                           'char_vocab_size': char_vocab.num_chars}, 
             elmo_config = {'num_layers': 3,
                            'word_embedding_dim': 150,
                            'vocab_size': word_vocab.num_words}, 
             char_vocab_size = char_vocab.num_chars).to(device)

In [8]:
# load model
elmo.load_state_dict(torch.load('../ELMo/elmo_epoch_3.pt'))

<All keys matched successfully>

In [36]:
def train_sentiment(model, train_loader, val_loader, optimizer, criterion, epochs):
    model.train()
    model.to(device)
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        predictions = []
        targets = []
        predictions_val = []
        targets_val = []
        loss_val = 0
        for batch in tqdm(train_loader):
            optimizer.zero_grad()
            sentences, labels = batch
            sentences = sentences.to(device)
            labels = labels.to(device)
            output = model(sentences)
            onehot_labels = torch.nn.functional.one_hot(labels, num_classes=2).float()
            loss = criterion(output, onehot_labels)
            loss.backward()
            optimizer.step()   
            total_loss += loss.item()
            predictions.extend(torch.argmax(output, dim=1).tolist())
            targets.extend(labels.tolist())
        model.eval()
        with torch.no_grad():
            for batch in val_loader:
                sentences, labels = batch
                sentences = sentences.to(device)
                labels = labels.to(device)
                output = model(sentences)
                onehot_labels = torch.nn.functional.one_hot(labels, num_classes=2).float()
                loss = criterion(output, onehot_labels)
                predictions_val.extend(torch.argmax(output, dim=1).tolist())
                targets_val.extend(labels.tolist())
                loss_val += loss.item()

        wandb.log({"Train Loss": total_loss/len(train_loader), 
                   "Train Accuracy": torch.sum(torch.tensor(predictions) == torch.tensor(targets)).item()/len(targets), 
                   "Val Accuracy": torch.sum(torch.tensor(predictions_val) == torch.tensor(targets_val)).item()/len(targets_val)})
        print(f"Epoch {epoch+1}")
        print(f"Train Loss: {total_loss/len(train_loader)}")
        print(f"Train Accuracy: {torch.sum(torch.tensor(predictions) == torch.tensor(targets)).item()/len(targets)}")  
        print(f"Val Accuracy: {torch.sum(torch.tensor(predictions_val) == torch.tensor(targets_val)).item()/len(targets_val)}")


In [84]:
dataset = SentimentAnalysisDataset(data, labels, word_vocab, char_vocab)
# split into train and val
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=dataset.collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=True, collate_fn=dataset.collate_fn)

In [85]:
sentiment_analysis = SentimentAnalysis(elmo, 300, 2).to(device)
optimizer = torch.optim.Adam(sentiment_analysis.parameters(), lr=0.0005)
criterion = nn.CrossEntropyLoss()

In [86]:
config = {'model': 'ELMo', 'language': 'Marathi', 'epochs': 5, 'batch_size': 32, 'lr': 0.0005}

wandb.init(project='INLP-Project-ELMo', group='Sentiment Analysis', name='Marathi with ELMo', config=config)
wandb.watch(sentiment_analysis)
train_sentiment(sentiment_analysis, train_dataloader, val_dataloader, optimizer, criterion, 5)  
# end wandb run
wandb.join()

  0%|          | 0/60 [00:00<?, ?it/s]/home2/sanika/inlp/INLP-Project-ELMo/Sentiment_Analysis/sentiment_preprocessing.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return [torch.tensor([self.char_vocab.char_to_index(char) for char in word], dtype=torch.long) for word in sentence], torch.tensor(self.labels[idx], dtype=torch.long)
100%|██████████| 60/60 [00:03<00:00, 19.44it/s]


Epoch 1
Train Loss: 0.6826742847760519
Train Accuracy: 0.5620743844944998
Val Accuracy: 0.6192468619246861


100%|██████████| 60/60 [00:03<00:00, 18.10it/s]


Epoch 2
Train Loss: 0.5674161990483602
Train Accuracy: 0.7213200628601362
Val Accuracy: 0.6841004184100419


100%|██████████| 60/60 [00:03<00:00, 18.87it/s]


...

Epoch 4
Train Loss: 0.24341179927190146
Train Accuracy: 0.9083289680460974
Val Accuracy: 0.7217573221757322


100%|██████████| 60/60 [00:03<00:00, 19.53it/s]


Epoch 5
Train Loss: 0.15980822313576937
Train Accuracy: 0.9418543740178104
Val Accuracy: 0.7196652719665272


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Train Accuracy,▁▄▆▇█
Train Loss,█▆▄▂▁
Val Accuracy,▁▅███
Train Accuracy,0.94185
Train Loss,0.15981
Val Accuracy,0.71967


In [87]:
dataset2 = EmbeddingDataset(data, labels, word_vocab)
# split into train and val
train_size = int(0.8 * len(dataset2))
val_size = len(dataset2) - train_size
train_dataset2, val_dataset2 = torch.utils.data.random_split(dataset2, [train_size, val_size])
train_dataloader2 = DataLoader(train_dataset2, batch_size=32, shuffle=True, collate_fn=dataset2.collate_fn)
val_dataloader2 = DataLoader(val_dataset2, batch_size=16, shuffle=True, collate_fn=dataset2.collate_fn)

In [88]:
sentiment_analysis_wo_elmo = SentimentAnalysis_WithoutELMo(300, 2, word_vocab).to(device)
optimizer = torch.optim.Adam(sentiment_analysis_wo_elmo.parameters(), lr=0.0005)
criterion = nn.CrossEntropyLoss()

In [89]:
config = {'model': 'No ELMo', 'language': 'Marathi', 'epochs': 5, 'batch_size': 32, 'lr': 0.0005}
wandb.init(project='INLP-Project-ELMo', group='Sentiment Analysis', name='Marathi without ELMo', config=config)

wandb.watch(sentiment_analysis_wo_elmo)
train_sentiment(sentiment_analysis_wo_elmo, train_dataloader2, val_dataloader2, optimizer, criterion, 5)  
# end wandb run
wandb.join()

  0%|          | 0/60 [00:00<?, ?it/s]/home2/sanika/inlp/INLP-Project-ELMo/Sentiment_Analysis/sentiment_preprocessing.py:77: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor([self.word_vocab.word_to_index(word) for word in self.data[idx]]), torch.tensor(self.labels[idx])
100%|██████████| 60/60 [00:01<00:00, 45.45it/s]


Epoch 1
Train Loss: 0.6840167939662933
Train Accuracy: 0.5500261917234154
Val Accuracy: 0.5794979079497908


100%|██████████| 60/60 [00:01<00:00, 52.51it/s]


Epoch 2
Train Loss: 0.6005390425523122
Train Accuracy: 0.6820324777370351
Val Accuracy: 0.6652719665271967


100%|██████████| 60/60 [00:01<00:00, 53.55it/s]


...

Epoch 4
Train Loss: 0.32939918811122576
Train Accuracy: 0.8653745416448402
Val Accuracy: 0.6841004184100419


100%|██████████| 60/60 [00:01<00:00, 50.71it/s]


Epoch 5
Train Loss: 0.19817009239147107
Train Accuracy: 0.9245678365636459
Val Accuracy: 0.694560669456067


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Train Accuracy,▁▃▆▇█
Train Loss,█▇▅▃▁
Val Accuracy,▁▆▄▇█
Train Accuracy,0.92457
Train Loss,0.19817
Val Accuracy,0.69456
